In [ ]:
import pandas as pd 

In [ ]:
campaign = pd.read_csv("../csv/num_3/campaign_master.csv")
class_master = pd.read_csv("../csv/num_3/class_master.csv")
customer = pd.read_csv("../csv/num_3/customer_master.csv")
use = pd.read_csv("../csv/num_3/use_log.csv")

In [ ]:
customer_join = pd.merge(customer, class_master, on="class", how="left")
customer_join = pd.merge(customer_join, campaign, on='campaign_id', how="left")
customer_join.head(2)

In [ ]:
# 2019년 01월 01일 이후의 데이터를 출력
# start_date, end_date 2개의 컬럼의 데이터를 datetime형태로 변경
customer_join["start_date"] = pd.to_datetime(customer_join["start_date"])
customer_join["end_date"] = pd.to_datetime(customer_join["end_date"])
customer_start = customer_join.loc[customer_join["start_date"] > pd.to_datetime("20190101")]
customer_start.head(2)

In [ ]:
## 데이터 집계
customer_start.value_counts("class_name")

In [ ]:
customer_start.groupby("campaign_name").count()["customer_id"]

In [ ]:
customer_start.value_counts("gender")

In [ ]:
use

In [ ]:
# 이용 이력 데이터를 이용하여 요일별 이용 여부를 계산
# 파생 변수 2개 생성
# weekday 요일 파생변수
# 년월 파생변수 생성
use["usedate"] = pd.to_datetime(use["usedate"])
use["년월"] = use["usedate"].dt.strftime("%Y%m")
use["weekday"] = use["usedate"].dt.weekday
use.value_counts("weekday")

In [ ]:
use_weekday = use.groupby(["customer_id", "년월", "weekday"], as_index=False).count()\
    [["customer_id", "년월", "weekday", "log_id"]]
# 컬럼의 이름을 변경하는 경우
use_weekday.rename(columns={"log_id" : "count"}, inplace=True)
use_weekday.head(10)

In [ ]:
# 각 회원별 가장 많은 count를 가지는 요일 출력
use_weekday = use_weekday.groupby("customer_id", as_index=False).max()\
    [["customer_id", "count"]]

In [ ]:
## count 값을 기준으로 4보다 높으면 1 아니면 0인 컬럼(routine_flg) 생성
use_weekday["routine_flg"] = use_weekday["count"].apply(lambda x : "1" if x >= 4 else "0")
use_weekday.head(10)

In [ ]:
## start_date 와 end_date사이의 기간 확인하는 라이브러리 
from dateutil.relativedelta import relativedelta

In [ ]:
## customer_join , use_weekday 데이터프레임을 join 결합
## customer_id을 조건으로 결합 use_weekday에 있는 routine_flg 컬럼만 결합
customer_join = pd.merge(customer_start, use_weekday[["customer_id", "routine_flg"]], 
on="customer_id", how="left")

In [ ]:
## gender 컬럼의 삭제
del customer_join["gender"]
customer_join.head()

In [ ]:
customer_join.isna().sum()

In [ ]:
customer_join.value_counts("end_date")

In [ ]:
customer_join["end_date"] = customer_join["end_date"].fillna(
    pd.to_datetime("20190430"))

In [ ]:
relativedelta(customer_join["end_date"].loc[0], 
customer_join["start_date"].loc[0])

In [ ]:
## 새로운 파생변수(membership_period) 생성
## 회원들의 기간을 월로 표시
len(customer_join)

In [ ]:
customer_join["membership_period"] = 0

for i in range(len(customer_join)):
    delta = relativedelta(customer_join["end_date"].loc[i], 
    customer_join["start_date"].loc[i])
    customer_join["membership_period"].loc[i] = \
        delta.years*12 + delta.months

In [ ]:
customer_join.value_counts("membership_period")

### 함수
- 행동 -> 반복적인 코드가 실행이 되야되는 경우에 코드를 반복해서 사용하면 코드 라인이 늘어나고 가독성이 떨어진다. 코드가 길어짐으로써 에러가 발생할 확률이 증가.
> def
- def 함수명 (매개변수)\
    함수 호출 시 실행이 될 코드 \
    return 반환되는 값
- 함수명(인자) --> 함수 호출

In [ ]:
# 함수에 매개변수 1개 
# 인자값이 숫자의 형태가 아니면 '숫자가 아니다'
# 숫자의 형태이면 짝수인 경우 '짝수' 
# 홀수인 경우에는 '홀수'

def func_1(x):
    if type(x) != int:
        return "숫자가 아니다"
    else:
        if x % 2 == 0:
            return "짝수"
        else :
            return "홀수"

print(func_1(4))    #짝수
print(func_1(3))    #홀수
print(func_1("asd"))#숫자가 아니다
print(func_1("3"))  #숫자가 아니다
print(func_1(3.1))

In [ ]:
## 함수 생성
## 매개변수의 값을 1개 
## 인자 값에 리스트 형태의 데이터 삽입
## 리턴 리스트의 순서를 반대로 바꿔서 출력

def func_2(x):
    # return list(reversed(x))
    x.reverse()
    return x

func_2([1,2,3,4,5])

In [ ]:
x = [1,2,3,4,5]

print(x[0], x[1], x[2], x[3], x[4])

In [ ]:
## range(시작값, 종료값, 증감값) --> [4,3,2,1,0]
y = []
for i in range(4, -1, -1):
    y.append(x[i])
y

In [ ]:
def func_3(x):
    result = []
    for i in range(len(x)-1, -1, -1):
        result.append(x[i])
    return result

func_3([1,2,3])

In [ ]:
def func_4(x):
    result = []
    for i in range(len(x), 0, -1):
        result.append(x[i-1])
    return result

func_4([1,2,3,4,2,3,1])

In [ ]:
## 함수 생성
## 매개변수 2개 숫자형 데이터 
## 두 수 사이 값들의 합 리턴
## a = 3, b = 5 인 경우 3+4+5 = 12 -> 12가 리턴이 되는 형식
## a와 b의 대소관계는 정해져있지 않다.

def func_5(a, b):
    sum = 0
    if a < b:
        for i in range(a, b+1, 1):
            sum += i
    else:
        for i in range(b, a+1, 1):
            sum += i
    return sum

print(func_5(1, 10))
print(func_5(10, 1)) 

In [ ]:
def func_6(a, b):
    result = range(min(a,b), max(a,b)+1, 1)
    return sum(result)

print(func_6(1, 10))
print(func_6(10, 1))

In [ ]:
def func_7(a, b):
    sum = 0
    if a < b:
        while a < b+1:
            sum +=a
            a+=1
    else : 
        while b < a+1:
            sum += b
            b += 1
    return sum
func_7(10, 1)

In [ ]:
## 함수 생성
## 매개변수 2개 
## x값부터 시작해서 x만큼 증가, 개수 y인 리스트 리턴
## 2,5 -> [2,4,6,8,10]

def func_8(x, y):
    result = list(range(x, x*(y+1), x))

    return result
func_8(-2,5)

In [ ]:
def func_9(x, y):
    result = []
    for i in range(y):
        result.append(x + x*i)
    return result
func_9(2,5)

In [99]:
## 자리수로 리스트 만드는 방법
a = 123456

In [103]:
for i in str(a):
    print(i)

1
2
3
4
5
6


In [102]:
str(a)[3]

'4'

In [105]:
## 함수 생성
## 매개변수 1개
## 숫자형 데이터를 받아와서 자릿수 합계
## 123 -> 1+2+3 =6 6을 리턴

def func_10(a):
    sum = 0
    for i in str(a):
        sum += int(i)
    return sum
func_10(12345)

15

## 변수의 종류
> 전역변수
- 어디서든 사용할 수 있는 변수
> 지역변수
- 특정 영역에서만 사용할 수 있는 변수

In [109]:
sum = 0
def test(a):
    result = 2
    print(result)
    return result
## 위의 코드에서 전역 변수 : sum
## 지역 변수 : result
print(sum)
test(1)

0
2


2

In [157]:
sum = 0
def test(a):
    global sum
    sum += a
    return sum


In [172]:
test(5)

75

In [194]:
## 비어있는 리스트 만들어서 
## 함수를 실행할때마다 리스트에 값이 추가가 되는 함수를 생성

_list = []
def test_2(a):
    _list.append(a)
    return _list

In [206]:
test_2(1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]